# Seepage analysis using Finite Difference Method


## Gauss-Seidel iteration

In [ ]:
import numpy as np

import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline

# Convolution
from scipy.ndimage import convolve, generate_binary_structure

# Geometry
## Dam
ldam = 200
rdam = 240

## Head
h = np.zeros([80, 440])
lhead = 105
rhead = 80

# Left head
h[0, 0:ldam]= lhead
# Right head
h[0, (ldam + 1):]= rhead

# Error and tolerance
tolerance = 1.E-9
error = 1.0
step = 0

# Residual
residual = np.zeros([80, 440])

(nrows, ncols) = h.shape

# Gauss-Seidel iteration
while error > tolerance:
    step = step + 1
    # Copy the head grid to residual
    residual = h.copy()
    
    for i in range(0, nrows):
        for j in range(0, ncols):
            # Top left side of dam boundary assign head
            if i == 0 and j <= (ldam + 1):
                h[i,j] = lhead
            
            # Top boundary beneath the dam
            elif i == 0 and j > (ldam + 1) and j < (rdam + 1):
                h[i,j] = (h[i,j-1]+h[i,j+1]+2*h[i+1,j])/4;
            
            # Top right side of dam boundary assign head
            elif i == 0 and j > (rdam):
                h[i,j]= rhead
                
            # Left boundary
            elif j == 0 and i > 0 and i < (nrows -1):
                h[i,j]=(h[i-1,j]+h[i+1,j]+2*h[i,j+1])/4

            # Right boundary
            elif j == (ncols -1) and i > 0 and i < (nrows - 1):
                h[i,j]=(2*h[i,j-1]+h[i-1,j]+h[i+1,j])/4
            
            # Left bottom corner
            elif i == (nrows -1) and j == 0:
                h[i,j]=(h[i-1,j]+h[i,j+1])/2
            
            # Bottom boundary
            elif i == (nrows -1) and j > 0 and j < (ncols - 1):
                h[i,j]=(h[i,j-1]+h[i,j+1]+2*h[i-1,j])/4
            
            # Right bottom corner
            elif i == (nrows -1) and  j == (ncols -1):
                h[i,j]=(h[i,j-1]+h[i-1,j])/2
            
            # Domain
            else:
                h[i,j]=(h[i-1,j]+h[i+1,j]+h[i,j-1]+h[i,j+1])/4
    
    # Error
    error = (np.square(np.subtract(h, residual))).max()

print('# of steps: %d, error: %le' %(step, error))

In [ ]:
plt.contour(np.flipud(h), levels=25)

## Jacobi iteration with convolution

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Convolution
from scipy.ndimage import convolve, generate_binary_structure
import time

# Geometry
rows = 80
cols = 440

## Dam
ldam = 200
rdam = 240

## Head
h = np.zeros([rows, cols])
lhead = 105
rhead = 80

# Error and tolerance
tolerance = 1.E-9
error = 1.0
step = 0

# Residual
residual = np.zeros([80, 440])

(nrows, ncols) = h.shape

# Start clock
start_time = time.time()

# Jacobi iteration
while error > tolerance:
    # Number of steps
    step = step + 1
    
    # Copy the head grid to residual
    residual = h.copy()
    
    # Finite Difference Method
    # Top left side of dam boundary assign head
    # if i == 0 and j <= (ldam):
    h[0, :(ldam+1)] = lhead

    # Top right side of dam boundary assign head
    # i == 0 and j > (rdam):
    h[0, (rdam+1):]= rhead
    
    # h[i, j] = (h[i-1, j] + h[i, j-1] + h[i, j+1] + h[i+1, j]) / 4
    # Define a kernel function with active neighbours
    k = generate_binary_structure(2, 1)
    k[1, 1] = False
    
    # Use convolution instead of loop iteration
    # https://en.wikipedia.org/wiki/Convolution
    B = convolve(h, k, mode='constant', cval=0.0)
    B *= 0.25
    
    # Top boundary beneath the dam
    # i == 0 and j > (ldam + 1) and j < (rdam + 1):
    for j in range(ldam+1, rdam+1):
        B[0, j] = (h[0,j-1]+h[0,j+1]+2*h[1,j])/4;
    
    # Left boundary
    # j == 0 and i > 0 and i < (nrows -1):
    for i in range(1, (nrows - 1)):
        B[i, 0] = (h[i-1, 0] + h[i+1, 0] + 2 * h[i, 1])/4

    # Right boundary
    # j == (ncols -1) and i > 0 and i < (nrows - 1):
    for i in range(1, (nrows - 1)):
        B[i, (ncols - 1)] = (2 * h[i, ncols-2] + h[i-1, ncols - 1] + h[i+1, ncols - 1]) / 4
            
    # Left bottom corner
    # i == (nrows -1) and j == 0:
    B[(nrows - 1), 0] = (h[(nrows - 2), 0] + h[(nrows - 1), 1]) / 2
            
    # Bottom boundary
    # i == (nrows -1) and j > 0 and j < (ncols - 1):
    for j in range(1, (ncols - 1)):
        B[(nrows -1), j] = (h[(nrows - 1), j-1] + h[(nrows - 1), j+1] + 2 * h[(nrows - 2), j])/4
            
    # Right bottom corner
    # i == (nrows -1) and  j == (ncols -1):
    B[(nrows - 1), (ncols - 1)] = (h[(nrows - 1), (ncols -2)] + h[(nrows -2), (ncols -1)])/2
    
    h = B.copy()
    
    # Calculate error
    error = (np.square(np.subtract(h, residual))).max()

elapsed_time = time.time() - start_time    
print('# of steps: %d, duration: %d seconds, error: %le' %(step, elapsed_time, error))

# of steps: 63385, duration: 96 seconds, error: 9.998125e-10


In [1]:
plt.contourf(np.flipud(h), levels=25)

NameError: name 'plt' is not defined

## Naive Jacobi

In [ ]:
import numpy as np

import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline

# Convolution
from scipy.ndimage import convolve, generate_binary_structure

# Geometry
## Dam
ldam = 200
rdam = 240

## Head
h = np.zeros([80, 440])
lhead = 105
rhead = 80

# Error and tolerance
tolerance = 1.E-9
error = 1.0
step = 0

# Residual
residual = np.zeros([80, 440])

(nrows, ncols) = h.shape

# Naive Jacobi iteration
while error > tolerance:
    step = step + 1
    # Copy the head grid to residual
    residual = h.copy()
    
    b = np.zeros([80, 440])
    for i in range(0, nrows):
        for j in range(0, ncols):
            # Top left side of dam boundary assign head
            if i == 0 and j <= ldam:
                b[i,j] = lhead
            
            # Top boundary beneath the dam
            elif i == 0 and j > ldam and j < (rdam + 1):
                b[i,j] = (h[i,j-1]+h[i,j+1]+2*h[i+1,j])/4;
            
            # Top right side of dam boundary assign head
            elif i == 0 and j > (rdam):
                b[i,j]= rhead
            
            # Left boundary
            elif j == 0 and i > 0 and i < (nrows -1):
                b[i,j]=(h[i-1,j]+h[i+1,j]+2*h[i,j+1])/4

            # Right boundary
            elif j == (ncols -1) and i > 0 and i < (nrows - 1):
                b[i,j]=(2*h[i,j-1]+h[i-1,j]+h[i+1,j])/4
            
            # Left bottom corner
            elif i == (nrows -1) and j == 0:
                b[i,j]=(h[i-1,j]+h[i,j+1])/2
            
            # Bottom boundary
            elif i == (nrows -1) and j > 0 and j < (ncols - 1):
                b[i,j]=(h[i,j-1]+h[i,j+1]+2*h[i-1,j])/4
            
            # Right bottom corner
            elif i == (nrows -1) and  j == (ncols -1):
                b[i,j]=(h[i,j-1]+h[i-1,j])/2
            
            # Domain
            else:
                b[i,j]=(h[i-1,j] + h[i+1,j]+h[i,j-1]+h[i,j+1])/4
    
    h = b.copy()
    # Error
    error = (np.square(np.subtract(h, residual))).max()

print('# of steps: %d, error: %le' %(step, error))
plt.contour(np.flipud(h), levels=10)